In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import datetime as date

In [5]:

data = pd.read_excel("Taux_Remplissage_ComCom_Nebbiu.xlsx")
data = data.drop(['Unnamed: 0'], axis=1)
dates = data['Unnamed: 1']
data.head()

,Unnamed: 1,poubelle 1,Unnamed: 3,Unnamed: 4,poubelle 2,Unnamed: 6,Unnamed: 7,poubelle 3,Unnamed: 9,Unnamed: 10,...,Unnamed: 13,poubelle 5,Unnamed: 15,Unnamed: 16,poubelle 6,Unnamed: 18,Unnamed: 19,poubelle 7,Unnamed: 21,Unnamed: 22
0,date,GPS,remplissage,coeff touriste,GPS,remplissage,coeff touriste,GPS,remplissage,coeff touriste,...,coeff touriste,GPS,remplissage,coeff touriste,GPS,remplissage,coeff touriste,GPS,remplissage,coeff touriste
1,2023-01-01 00:00:00,"42°36'39""N 9°21'14""E",5,1,"42°36'44""N 9°21'24""E",18,1,"42°36'51""N 9°21'17""E",21,1,...,1,"42°37'01""N 9°21'20""E",9,1,"42°36'30""N 9°21'29""E",13,1,"42°36'07""N 9°21'35""E",11,1
2,2023-01-02 00:00:00,NaN,40,1,NaN,51,1,NaN,46,1,...,1,NaN,51,1,NaN,45,1,NaN,40,1
3,2023-01-03 00:00:00,NaN,0,1,NaN,0,1,NaN,0,1,...,1,NaN,0,1,NaN,0,1,NaN,0,1
4,2023-01-04 00:00:00,NaN,37,1,NaN,25,1,NaN,30,1,...,1,NaN,38,1,NaN,42,1,NaN,49,1


In [6]:
def create_df(num_poubelle: int) -> pd.DataFrame:
    df = pd.DataFrame()
    df['date'] = dates[1:]
    df['remplissage'] = data[f"Unnamed: {num_poubelle*3}"][1:]
    df['coeff'] = data[f"Unnamed: {num_poubelle*3+1}"][1:]
    df = df.dropna()
    return df


df = create_df(1)

print(df.head())


def train_model(df: pd.DataFrame, date_: date.datetime) -> LinearRegression:
    for i in range(3, 6):
        df[f'remplissage_{i}d_ago'] = df['remplissage'].shift(i)

    X = df.drop(['date', 'remplissage'], axis=1)
    y = df['remplissage']

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_scaled, y_train)

    predictions = model.predict(X_test_scaled)

    mse = mean_squared_error(y_test, predictions)
    print(f'Mean Squared Error: {mse}')

    future_date = pd.to_datetime(date_)
    features_future = []

    for i in range(3, 6):
        features_future.append(
            df.loc[df['date'] == future_date - date.timedelta(days=i), 'remplissage'].values[0])

    features_future_scaled = scaler.transform([features_future + [0]])
    prediction_future = model.predict(features_future_scaled)

    features_future_scaled = scaler.transform([features_future + [0]])
    prediction_future = model.predict(features_future_scaled)

    last_emptied_index = df[df['remplissage'] == 0].index[-1]
    last_emptied_date = df.loc[last_emptied_index, 'date']
    growth_rate = (prediction_future[0] - df.loc[last_emptied_index,
                   'remplissage']) / (future_date - last_emptied_date).days

    if prediction_future[0] > 75 or growth_rate > 10:
        print(
            f'The trash can is worth emptying. Predicted Remplissage for {future_date}: {prediction_future[0]}')
    else:
        print(
            f'The trash can is not worth emptying. Predicted Remplissage for {future_date}: {prediction_future[0]}')
        
    return model

train_model(df, date.datetime(2021, 5, 1))

                  date remplissage coeff
1  2023-01-01 00:00:00           5     1
2  2023-01-02 00:00:00          40     1
3  2023-01-03 00:00:00           0     1
4  2023-01-04 00:00:00          37     1
5  2023-01-05 00:00:00          79     1


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

remind that when the remplissage is =0 it's becuase the garbage man had empty the trash can so the algorithme should predict if will be full today , base on the rates of increse between the last time it was emptied (the first remplissage =0 in the last 5 days) and the coeff (1 we check the rates, 2 once every 2 days, 3 everydays ), the trash can is considered worth to be emptied if it's content is > 75 and/or it rates of growth is >10 a days